In [2]:
from sdv.metadata import Metadata
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
df_m = "df_min"
df_majority = "df_majority"

selected_file = df_m
files = [df_m, df_majority]
df= pd.read_csv("../dataset/" + selected_file + ".csv")
df_fake = pd.read_csv("../dataset/fake_" + selected_file + ".csv")


In [5]:
df= df.drop(columns=["Unnamed: 0"])
df.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,1,181.00,181.00,0.0,21182.00,0.00
1,1,2806.00,2806.00,0.0,26202.00,0.00
2,1,20128.00,20128.00,0.0,6268.00,12145.85
3,1,25071.46,25071.46,0.0,9083.76,34155.22
4,1,235238.66,235238.66,0.0,0.00,235238.66


In [6]:
df_fake.head()

,type,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
0,4,479982.30,792001.28,0.00,0.00,37578.33
1,1,44449.46,47688.89,0.00,29086.87,242051.44
2,4,982102.67,602964.14,0.00,0.00,0.00
3,4,51395.55,142310.86,0.00,25178.16,0.00
4,4,93283.56,101835.02,7912.79,33998.11,43772.83


This metric measures the correlation between a pair of numerical columns and computes the similarity between the real and synthetic data -- aka it compares the trends of 2D distributions.

Let's try amount and newbalanceDest
If all clusters results was high -> i have totally noisy random distribution

In [7]:
from sdmetrics.column_pairs import CorrelationSimilarity
correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    correlation_similarity = CorrelationSimilarity.compute(
    real_data=df_org[["amount","newbalanceDest"]],
    synthetic_data=df_fake[["amount","newbalanceDest"]],
    coefficient='Spearman'
    )
    print(correlation_similarity)
    correlation_similarities.append(correlation_similarity)


df_min
0.8868432187994814
df_majority
0.994517328976503


KSComplement, This metric computes the similarity of a real column vs. a synthetic column in terms of the column shapes -- aka the marginal distribution or 1D histogram of the column.

marginal distribution or 1D histogram

In [8]:
from sdmetrics.single_column import KSComplement
correlation_similarities = []
column_scores = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    for column in df_org.columns:
        if column in ["type"]:
            continue
        ks_comp = KSComplement.compute(
        real_data=df_org[column],
        synthetic_data=df_fake[column],
        )
        # print(column)
        # print(ks_comp)
        column_scores.append(ks_comp)

    avg = sum(column_scores)/len(column_scores)
    print(avg)
    correlation_similarities.append(avg)
    column_scores = []


df_min
0.776914647510045
df_majority
0.7889268292682926


StatisticSimilarity: This metric measures the similarity between a real column and a synthetic column by comparing a summary statistic. Supported summary statistics are: mean, median and standard deviation.


In [9]:
import math
from sdmetrics.single_column import StatisticSimilarity
correlation_similarities = []
column_scores = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    for column in df_org.columns:
        if column in ["type"]:
            continue
        ks_comp = StatisticSimilarity.compute(
        real_data=df_org[column],
        synthetic_data=df_fake[column],
        statistic="std"
        )
        if math.isnan(ks_comp):
            continue
        # print(column)
        # print(ks_comp)
        column_scores.append(ks_comp)

    avg = sum(column_scores)/len(column_scores)
    print(avg)
    correlation_similarities.append(avg)
    column_scores = []


df_min
0.989564185181751
df_majority
0.9939953072188615


TVComplement:
This metric computes the similarity of a real column vs. a synthetic column in terms of the column shapes -- aka the marginal distribution or 1D histogram of the column.
Only for categorical data

In [10]:
from sdmetrics.single_column import TVComplement
correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = TVComplement.compute(
    real_data=df_org["type"],
    synthetic_data=df_fake["type"],
    )

    print(ks_comp)


df_min
0.9929380250821868
df_majority
0.9523170731707318


Privacy metrics

The DCRBaselineProtection metric measures the distance between your synthetic data and real data to measure how private it is. For a fair measurement, it compares the distance against randomly generated data, which would provide the best possible privacy protection.

DCR = Distance to Closest Record


In [11]:
from sdmetrics.single_table import DCRBaselineProtection
from sdv.metadata import Metadata

correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = DCRBaselineProtection.compute_breakdown(
    real_data=df_org,
    synthetic_data=df_fake,
    metadata= Metadata.detect_from_dataframe(df_org).to_dict().get("tables").get("table")
    )

    print(ks_comp['score'])


df_min
0.0028982807629885535
df_majority
0.003173781414729576


(best) 1.0: The synthetic data has the highest possible privacy protection. Replacing the synthetic data entirely with random data would not improve the privacy.

(worst) 0.0: The synthetic has the worst possible privacy protection. Compared to random data, the synthetic data is much closer to the real data.

The DisclosureProtection metric measures the risk associated with disclosing (aka broadly sharing) the synthetic data. It's a useful measurement if you want to know whether synthetic data is leaking patterns that pertain to sensitive information.

In [12]:
from sdmetrics.single_table import DisclosureProtection
from sdv.metadata import Metadata

correlation_similarities = []
for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    ks_comp = DisclosureProtection.compute(
    real_data=df_org,
    synthetic_data=df_fake,
    known_column_names=['newbalanceOrig','newbalanceDest','oldbalanceDest'],
    sensitive_column_names=['amount'],
    )

    print(ks_comp)


df_min
1
df_majority
1


The whole quality report

The Single Table Quality Report evaluates how well your synthetic data captures mathematical properties in your data.

AVG between Evaluating Column Shapes and Evaluating Column Pair Trends for each coloumn


In [13]:
from sdmetrics.reports.single_table import QualityReport

report = QualityReport()

for file in files:
    df_org = pd.read_csv("../dataset/" + file + ".csv")
    df_org = df_org.drop(columns=["Unnamed: 0"])
    df_fake = pd.read_csv("../dataset/fake_" + file + ".csv")
    print(file)
    report.generate(df_org, df_fake, Metadata.detect_from_dataframe(df_org).to_dict().get("tables").get("table"),verbose=False)
    print(report.get_score())
    # print(report.get_details(property_name='Column Shapes'))


df_min
0.8575771811363007
df_majority
0.8496341755874961
